In [0]:
!wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/monet2photo.zip

--2019-08-22 12:55:44--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/monet2photo.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305231073 (291M) [application/zip]
Saving to: ‘monet2photo.zip’

monet2photo.zip     100%[===================>] 291.09M  21.2MB/s    in 15s     

2019-08-22 12:55:59 (19.6 MB/s) - ‘monet2photo.zip’ saved [305231073/305231073]



In [0]:
!mkdir data
!cd data

!unzip monet2photo.zip

Archive:  monet2photo.zip
   creating: monet2photo/
   creating: monet2photo/trainA/
  inflating: monet2photo/trainA/01159.jpg  
  inflating: monet2photo/trainA/01048.jpg  
  inflating: monet2photo/trainA/01144.jpg  
  inflating: monet2photo/trainA/00799.jpg  
  inflating: monet2photo/trainA/00897.jpg  
  inflating: monet2photo/trainA/00998.jpg  
  inflating: monet2photo/trainA/00883.jpg  
  inflating: monet2photo/trainA/00985.jpg  
  inflating: monet2photo/trainA/00311.jpg  
  inflating: monet2photo/trainA/01243.jpg  
  inflating: monet2photo/trainA/01259.jpg  
  inflating: monet2photo/trainA/00454.jpg  
  inflating: monet2photo/trainA/00461.jpg  
  inflating: monet2photo/trainA/01283.jpg  
  inflating: monet2photo/trainA/00901.jpg  
  inflating: monet2photo/trainA/00502.jpg  
  inflating: monet2photo/trainA/00176.jpg  
  inflating: monet2photo/trainA/01157.jpg  
  inflating: monet2photo/trainA/00935.jpg  
  inflating: monet2photo/trainA/00929.jpg  
  inflating: monet2photo/trainA/008

In [0]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL

from glob import glob
from keras import Input, Model
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, BatchNormalization, Activation
from keras.layers import Add, Conv2DTranspose, ZeroPadding2D, LeakyReLU
from keras.optimizers import Adam
from imageio import imread
from skimage.transform import resize

Using TensorFlow backend.


In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-s_z8f_t6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-s_z8f_t6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=cfc88de5fa2eac3e2695288ecdf4f8de4699b739e42455ffe03a614c72c4b121
  Stored in directory: /tmp/pip-ephem-wheel-cache-pzwn1cn5/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

## Residual Block

In [0]:
def residual_block(x):
  
  """
  Residual block
  """
  
  res = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same")(x)
  res = BatchNormalization(axis = 3, momentum = 0.9, epsilon = 1e-5)(res)
  res = Activation('relu')(res)
  res = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same")(res)
  res = BatchNormalization(axis = 3, momentum = 0.9, epsilon = 1e-5)(res)
  
  return Add()([res, x])

## Generator Network

In [0]:
def build_generator():
  
  """
  Creating a generator network with the hyperparameters defined below
  """
  
  input_shape = (128, 128, 3)
  residual_blocks = 6
  input_layer = Input(shape = input_shape)
  
  
  ## 1st Convolutional Block
  x = Conv2D(filters = 32, kernel_size = 7, strides = 1, padding = "same")(input_layer)
  x = InstanceNormalization(axis = 1)(x)
  x = Activation("relu")(x)
  
  ## 2nd Convolutional Block
  x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = "same")(x)
  x = InstanceNormalization(axis = 1)(x)
  x = Activation("relu")(x)
  
  ## 3rd Convolutional Block
  x = Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = "same")(x)
  x = InstanceNormalization(axis = 1)(x)
  x = Activation("relu")(x)
  
  
  ## Residual blocks
  for _ in range(residual_blocks):
    x = residual_block(x)
  
  
  ## 1st Upsampling Block
  x = Conv2DTranspose(filters = 64, kernel_size = 3, strides = 2, padding = "same", use_bias = False)(x)
  x = InstanceNormalization(axis = 1)(x)
  x = Activation("relu")(x)
  
  ## 2nd Upsampling Block
  x = Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2, padding = "same", use_bias = False)(x)
  x = InstanceNormalization(axis = 1)(x)
  x = Activation("relu")(x)
  
  ## Last Convolutional Layer
  x = Conv2D(filters = 3, kernel_size = 7, strides = 1, padding = "same")(x)
  output = Activation("tanh")(x)
  
  
  model = Model(inputs = [input_layer], outputs = [output])
  return model

## Discriminator Network

In [0]:
def build_discriminator():
  
  """
  Create a discriminator network using the hyperparameters defined below
  """
  
  input_shape = (128, 128, 3)
  hidden_layers = 3
  
  input_layer = Input(shape = input_shape)
  
  x = ZeroPadding2D(padding = (1, 1))(input_layer)
  
  
  ## 1st Convolutional Block
  x = Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "valid")(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = ZeroPadding2D(padding = (1, 1))(x)
  
  
  ## 3 Hidden Convolutional Blocks
  for i in range(1, hidden_layers + 1):
    x = Conv2D(filters = 2 ** i * 64, kernel_size = 4, strides = 2, padding = "valid")(x)
    x = InstanceNormalization(axis = 1)(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = ZeroPadding2D(padding = (1, 1))(x)
    
  
  ## Last Convolutional Layer
  output = Conv2D(filters = 1, kernel_size = 4, strides = 1, activation = "sigmoid")(x)
  
  
  model = Model(inputs = [input_layer], outputs = [output])
  return model

In [0]:
def load_images(data_dir):
  
  imagesA = glob(data_dir + '/testA/*.*')
  imagesB = glob(data_dir + '/testB/*.*')
  
  allImagesA = []
  allImagesB = []
  
  for index, filename in enumerate(imagesA):
    imgA = imread(filename, pilmode = "RGB")
    imgB = imread(imagesB[index], pilmode = "RGB")
    
    imgA = resize(imgA, (128, 128))
    imgB = resize(imgB, (128, 128))
    
    if np.random.random() > 0.5:
      imgA = np.fliplr(imgA)
      imgB = np.fliplr(imgB)
      
    
    allImagesA.append(imgA)
    allImagesB.append(imgB)
    
  
  ## Normalize images
  allImagesA = np.array(allImagesA) / 127.5 - 1.
  allImagesB = np.array(allImagesB) / 127.5 - 1.
  
  return allImagesA, allImagesB

In [0]:
def load_test_batch(data_dir, batch_size):
  
  imagesA = glob(data_dir + '/testA/*.*')
  imagesB = glob(data_dir + '/testB/*.*')
  
  imagesA = np.random.choice(a = imagesA, size = batch_size)
  imagesB = np.random.choice(a = imagesB, size = batch_size)
  
  allA = []
  allB = []
  
  for i in range(len(imagesA)):
    ## Load and resize images
    imgA = resize(imread(imagesA[i], pilmode = 'RGB').astype(np.float32), (128, 128))
    imgB = resize(imread(imagesB[i], pilmode = 'RGB').astype(np.float32), (128, 128))
    
    allA.append(imgA)
    allB.append(imgB)
    
  return np.array(allA) / 127.5 - 1.0, np.array(allB) / 127.5 - 1.0

In [0]:
!mkdir results

def save_images(originalA, generatedB, reconstructedA,
                originalB, generatedA, reconstructedB, path):
  
  """
  Save images
  """
  
  fig = plt.figure()
  
  ax = fig.add_subplot(2, 3, 1)
  ax.imshow(originalA)
  ax.axis("off")
  ax.set_title("Original")
  
  ax = fig.add_subplot(2, 3, 2)
  ax.imshow(generatedB)
  ax.axis("off")
  ax.set_title("Generated")
  
  ax = fig.add_subplot(2, 3, 3)
  ax.imshow(reconstructedA)
  ax.axis("off")
  ax.set_title("Reconstructed")
  
  ax = fig.add_subplot(2, 3, 4)
  ax.imshow(originalB)
  ax.axis("off")
  ax.set_title("Original")
  
  ax = fig.add_subplot(2, 3, 5)
  ax.imshow(generatedA)
  ax.axis("off")
  ax.set_title("Generated")
  
  ax = fig.add_subplot(2, 3, 6)
  ax.imshow(reconstructedB)
  ax.axis("off")
  ax.set_title("Reconstructed")
  
  plt.savefig(path)

In [0]:
def write_log(callback, name, loss, batch_no):
  
  """
  Write training summary to TensorBoard
  """
  
  summary = tf.Summary()
  summary_value = summary.value.add()
  summary_value.simple_value = loss
  summary_value.tag = name
  callback.writer.add_summary(summary, batch_no)
  callback.writer.flush()

In [0]:
if __name__ == '__main__':
  
  data_dir = "monet2photo"
  batch_size = 1
  epochs = 500
  mode = 'train'
  
  if mode == 'train':
    
    """
    Load dataset
    """
    
    imagesA, imagesB = load_images(data_dir = data_dir)
    
    ## Define the common optimizer
    common_optimizer = Adam(0.002, 0.5)
    
    
    ## Build and compile discriminator networks
    discriminatorA = build_discriminator()
    discriminatorB = build_discriminator()
    
    discriminatorA.compile(loss = 'mse', 
                           optimizer = common_optimizer,
                           metrics = ['accuracy'])
    discriminatorB.compile(loss = 'mse',
                           optimizer = common_optimizer,
                           metrics = ['accuracy'])
    
    
    ## Build generator networks
    generatorA_to_B = build_generator()
    generatorB_to_A = build_generator()
    
    
    """
    Create an adversarial network
    """
    
    inputA = Input(shape = (128, 128, 3))
    inputB = Input(shape = (128, 128, 3))
    
    
    ## --> Generated images using both of the generator networks
    generatedB = generatorA_to_B(inputA)
    generatedA = generatorB_to_A(inputB)
    
    
    ## --> Reconstruct the images back to the original ones
    reconstructedA = generatorB_to_A(generatedB)
    reconstructedB = generatorA_to_B(generatedA)
    
    generatedA_Id = generatorB_to_A(inputA)
    generatedB_Id = generatorA_to_B(inputB)
    
    
    ## Make both of the discriminator networks non-trainable
    discriminatorA.trainable = False
    discriminatorB.trainable = False
    
    probsA = discriminatorA(generatedA)
    probsB = discriminatorB(generatedB)
    
    
    adversarial_model = Model(inputs = [inputA, inputB],
                              outputs = [probsA, probsB, 
                                         reconstructedA, reconstructedB,
                                         generatedA_Id, generatedB_Id])
    
    adversarial_model.compile(loss = ['mse', 'mse', 'mae', 'mae', 'mae', 'mae'],
                              loss_weights = [1, 1, 10.0, 10.0, 1.0, 1.0],
                              optimizer = common_optimizer)
    
    
    tensorboard = TensorBoard(log_dir = "logs/{}".format(time.time()),
                              write_images = True, write_grads = True,
                              write_graph = True)
    tensorboard.set_model(generatorA_to_B)
    tensorboard.set_model(generatorB_to_A)
    tensorboard.set_model(discriminatorA)
    tensorboard.set_model(discriminatorB)
    
    
    real_labels = np.ones((batch_size, 7, 7, 1))
    fake_labels = np.zeros((batch_size, 7, 7, 1))
    
    for epoch in range(epochs):
      print("Epoch: {}".format(epoch))
      
      D_losses = []
      G_losses = []
      
      num_batches = int(min(imagesA.shape[0], imagesB.shape[0]) / batch_size)
      print("Number of batches: {}".format(num_batches))
      
      
      for index in range(num_batches):
        print("Batch: {}".format(index))
        
        ## Sample images
        batchA = imagesA[index * batch_size: (index + 1) * batch_size]
        batchB = imagesB[index * batch_size: (index + 1) * batch_size]
        
        ## Translate images to opposite domain
        generatedB = generatorA_to_B.predict(batchA)
        generatedA = generatorB_to_A.predict(batchB)
        
        ## Train the discriminator A on real and fake images
        D_A_Loss1 = discriminatorA.train_on_batch(batchA, real_labels)
        D_A_Loss2 = discriminatorA.train_on_batch(generatedA, fake_labels)
        
        ## Train the discriminator B on real and fake images
        D_B_Loss1 = discriminatorB.train_on_batch(batchB, real_labels)
        D_B_Loss2 = discriminatorB.train_on_batch(generatedB, fake_labels)
        
        ## Calculate the total discriminator loss
        D_loss = 0.5 * np.add(0.5 * np.add(D_A_Loss1, D_A_Loss2), 
                              0.5 * np.add(D_B_Loss1, D_B_Loss2))
        
        print("D_Loss: {}".format(D_loss))
        
        
        """
        Train the generator networks
        """
        
        G_loss = adversarial_model.train_on_batch([batchA, batchB],
                                                  [real_labels, real_labels,
                                                   batchA, batchB,
                                                   batchA, batchB])
        
        print("G_Loss: {}".format(G_loss))
        
        D_losses.append(D_loss)
        G_losses.append(G_loss)
        
        
      """
      Save losses to TensorBoard after every epoch
      """
      
      write_log(tensorboard, 'discriminator_loss', np.mean(D_losses), epoch)
      write_log(tensorboard, 'generator_loss', np.mean(G_losses), epoch)
      
      
      ## Sample and save images after every 10 epochs
      if epoch % 10 == 0:
        
        ## Get a batch of test data
        batchA, batchB = load_test_batch(data_dir = data_dir, batch_size = 2)
        
        ## Generate images
        generatedB = generatorA_to_B.predict(batchA)
        generatedA = generatorB_to_A.predict(batchB)
        
        ## Get reconstructed images
        recons_A = generatorB_to_A.predict(generatedB)
        recons_B = generatorA_to_B.predict(generatedA)
        
        ## Save original, generated and reconstructed images
        for i in range(len(generatedA)):
          save_images(originalA = batchA[i], generatedB = generatedB[i], reconstructedA = recons_A[i],
                      originalB = batchB[i], generatedA = generatedA[i], reconstructedB = recons_B[i],
                      path = "results/gen_{}_{}".format(epoch, i))
          
    
    ## Save models
    generatorA_to_B.save_weights("generatorA_to_B.h5")
    generatorB_to_A.save_weights("generatorB_to_A.h5")
    discriminatorA.save_weights("discriminatorA.h5")
    discriminatorB.save_weights("discriminatorB.h5")
    
    
  elif mode == 'predict':
    
    ## Build generator networks
    generatorA_to_B = build_generator()
    generatorB_to_A = build_generator()
    
    generatorA_to_B.load_weights("generatorA_to_B.h5")
    generatorB_to_A.load_weights("generatorB_to_A.h5")
    
    
    ## Get a batch of test data
    batchA, batchB = load_test_batch(data_dir = data_dir, batch_size = 2)
    
    
    ## Save images
    generatedB = generatorA_to_B.predict(batchA)
    generatedA = generatorB_to_A.predict(batchB)
    
    reconsA = generatorB_to_A.predict(generatedB)
    reconsB = generatorA_to_B.predict(generatedA)
    
    for i in range(len(generatedA)):
      save_images(originalA = batchA[i], generatedB = generatedB[i], reconstructedA = recons_A[i],
                  originalB = batchB[i], generatedA = generatedA[i], reconstructedB = recons_B[i],
                  path = "results/test_{}".format(i))

W0822 12:57:49.734445 139716881598336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 12:57:49.736517 139716881598336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 12:57:49.752670 139716881598336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 12:57:50.060953 139716881598336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0822 12:57:50.235987 139716881598336 deprecation_wrappe

Epoch: 0
Number of batches: 121
Batch: 0


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


D_Loss: [0.5305587  0.49489796]
G_Loss: [25.205275, 5.0539395e-09, 6.623143e-08, 1.2187737, 1.0739226, 1.2182144, 1.0600963]
Batch: 1
D_Loss: [0.4999975 0.5      ]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

Epoch: 11
Number of batches: 121
Batch: 0
D_Loss: [0.5 0.5]
G_Loss: [1.0323367, 1.0, 0.0, 0.0011241152, 0.0018155951, 0.001124394, 0.0018151808]
Batch: 1
D_Loss: [0.5 0.5]
G_Loss: [1.0503628, 1.0, 0.0, 0.0018500803, 0.0027283058, 0.001850457, 0.002728514]
Batch: 2
D_Loss: [0.5 0.5]
G_Loss: [1.0312318, 1.0, 0.0, 0.0016515641, 0.0011877902, 0.0016505847, 0.0011876842]
Batch: 3
D_Loss: [0.5 0.5]
G_Loss: [1.0526011, 1.0, 0.0, 0.0029847133, 0.001797103, 0.0029860102, 0.0017969267]
Batch: 4
D_Loss: [0.5 0.5]
G_Loss: [1.0254378, 1.0, 0.0, 0.0011237985, 0.0011887223, 0.0011239868, 0.0011886555]
Batch: 5
D_Loss: [0.5 0.5]
G_Loss: [1.04115, 1.0, 0.0, 0.00096145994, 0.0027794505, 0.0009614271, 0.0027794938]
Batch: 6
D_Loss: [0.5 0.5]
G_Loss: [1.0618309, 1.0, 0.0, 0.0011945376, 0.004426442, 0.0011946002, 0.0044264365]
Batch: 7
D_Loss: [0.5 0.5]
G_Loss: [1.0339528, 1.0, 0.0, 0.0010948367, 0.0019918424, 0.0010941806, 0.001991741]
Batch: 8
D_Loss: [0.5 0.5]
G_Loss: [1.0352837, 1.0, 0.0, 0.0018823864,

W0822 13:12:00.810707 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:00.835649 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:00.855693 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


G_Loss: [1.039357, 1.0, 0.0, 0.0017186594, 0.0018592626, 0.0017186969, 0.0018590989]


W0822 13:12:00.874701 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:00.895823 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:00.913496 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:00.991259 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:01.010306 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:01.029375 139716881598336 image.py:648] Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
W0822 13:12:01.048117 139716

Epoch: 21
Number of batches: 121
Batch: 0
D_Loss: [0.5 0.5]
G_Loss: [1.0317351, 1.0, 0.0, 0.0010825285, 0.0018024929, 0.0010828323, 0.0018020737]
Batch: 1
D_Loss: [0.5 0.5]
G_Loss: [1.0511647, 1.0, 0.0, 0.0018909018, 0.0027604117, 0.0018909741, 0.002760621]
Batch: 2
D_Loss: [0.5 0.5]
G_Loss: [1.0320256, 1.0, 0.0, 0.0017160284, 0.0011955148, 0.0017146135, 0.0011955099]
Batch: 3
D_Loss: [0.5 0.5]
G_Loss: [1.0510559, 1.0, 0.0, 0.0028536946, 0.0017877315, 0.0028541593, 0.0017875144]
Batch: 4
D_Loss: [0.5 0.5]
G_Loss: [1.0260245, 1.0, 0.0, 0.0012029654, 0.0011628708, 0.0012032909, 0.0011627255]
Batch: 5
D_Loss: [0.5 0.5]
G_Loss: [1.0420709, 1.0, 0.0, 0.0010179211, 0.002806708, 0.0010178435, 0.0028068065]
Batch: 6
D_Loss: [0.5 0.5]
